## 1. 데이터 로딩

샘플로 1000 개의 뉴스 기사만 이용합니다. 

In [1]:
import sys
sys.path.append('../../../tutorial_data/')

from data_loader import get_news_corpus_as_list

docs = get_news_corpus_as_list(n_docs=1000)
docs = [doc for doc in docs if doc]
len(docs)

965

## 2. Build bigram dictionary

In [2]:
from sklearn.feature_extraction.text import CountVectorizer
from konlpy.tag import Komoran

komoran = Komoran()

bigram_extractor 는 min_count 이상인 bigram 만을 선택하는 extractor 입니다. 

to_bigram 은 두 개의 list 를 zip 으로 묶어서 bigram list 을 만드는 함수입니다. 

bigram_counter 로 카운팅을 한 뒤, min_count 이상인 bigram 만을 선택하여 bigram_dictionary 를 만듭니다. 총 2835 개의 bigram 을 선택하였습니다. 

In [3]:
from collections import Counter

def bigram_extractor(docs, min_count=10):

    def to_bigram(tokens):
        bigrams = [(t0, t1) for t0, t1 in zip(tokens, tokens[1:])]
        return bigrams

    bigram_counter = Counter(
        [bigram for doc in docs for bigram in
         to_bigram(komoran.pos(doc, join=True)) if doc
        ]
    )

    bigram_dictionary = {
        bigram:count for bigram, count in bigram_counter.items()
        if count >= min_count
    }

    return bigram_dictionary

bigrams = bigram_extractor(docs)
len(bigrams)

2835

임의로 다섯개의 bigram 을 살펴봅니다. 

In [4]:
list(bigrams)[:5]

[('서울/NNP', '연합뉴스/NNP'),
 ('연합뉴스/NNP', '경찰/NNG'),
 ('경찰/NNG', '관계자/NNG'),
 ('관계자/NNG', '들/XSN'),
 ('들/XSN', '이/JKS')]

## 3. Bigram tokenizer

BigramTokenizer 는 base tokenizer 인 tagger 와 bigrams 를 입력받는 class 입니다. 

\_\_call\_\_() 함수를 구현하면 클래스도 함수처럼 호출할 수 있습니다. 

In [5]:
class BigramTokenizer:

    def __init__(self, bigrams, tagger):
        self.bigrams = bigrams
        self.tagger = tagger

    def __call__(self, sent):
        if not sent:
            return []

        unigrams = self.tagger.pos(sent, join=True)

        bigrams = [(t0, t1) for t0, t1 in zip(unigrams, unigrams[1:])]
        bigrams = [bigram for bigram in bigrams if bigram in self.bigrams]
        bigrams = ['%s-%s' % (t0, t1) for t0, t1 in bigrams]

        return unigrams + bigrams

bigram_tokenizer = BigramTokenizer(bigrams, komoran)

샘플로 하나의 문장을 선택하여 토크나이징을 합니다. 

In [6]:
sent = docs[1].split('  ')[0]
sent

'오패산터널 총격전 용의자 검거 서울 연합뉴스 경찰 관계자들이 19일 오후 서울 강북구 오패산 터널 인근에서 사제 총기를 발사해 경찰을 살해한 용의자 성모씨를 검거하고 있다 성씨는 검거 당시 서바이벌 게임에서 쓰는 방탄조끼에 헬멧까지 착용한 상태였다 독자제공 영상 캡처 연합뉴스'

bigram_tokenizer(sent) 로 클래스를 호출합니다. 

In [7]:
bigram_tokenizer(sent)

['오패산터널/NNP',
 '총격전/NNG',
 '용의자/NNP',
 '검거/NNG',
 '서울/NNP',
 '연합뉴스/NNP',
 '경찰/NNG',
 '관계자/NNG',
 '들/XSN',
 '이/JKS',
 '19/SN',
 '일/NNB',
 '오후/NNG',
 '서울/NNP',
 '강북구/NNP',
 '오/NNP',
 '패사/NNG',
 'ㄴ/JX',
 '터널/NNP',
 '인근/NNG',
 '에서/JKB',
 '사제/NNP',
 '총기/NNG',
 '를/JKO',
 '발사/NNG',
 '하/XSV',
 '아/EC',
 '경찰/NNG',
 '을/JKO',
 '살해/NNG',
 '하/XSV',
 'ㄴ/ETM',
 '용의자/NNP',
 '성모/NNP',
 '씨/NNB',
 '를/JKO',
 '검거/NNG',
 '하/XSV',
 '고/EC',
 '있/VV',
 '다/EC',
 '성씨/NNP',
 '는/JX',
 '검거/NNG',
 '당시/NNG',
 '서바이벌/NNP',
 '게임/NNG',
 '에서/JKB',
 '쓰/VV',
 '는/ETM',
 '방탄/NNP',
 '조끼/NNP',
 '에/JKB',
 '헬멧/NNP',
 '까지/JX',
 '착용/NNG',
 '하/XSV',
 'ㄴ/ETM',
 '상태/NNG',
 '이/VCP',
 '었/EP',
 '다/EC',
 '독자/NNG',
 '제공/NNG',
 '영상/NNP',
 '캡처/NNP',
 '연합뉴스/NNP',
 '서울/NNP-연합뉴스/NNP',
 '연합뉴스/NNP-경찰/NNG',
 '경찰/NNG-관계자/NNG',
 '관계자/NNG-들/XSN',
 '들/XSN-이/JKS',
 '이/JKS-19/SN',
 '19/SN-일/NNB',
 '일/NNB-오후/NNG',
 '오후/NNG-서울/NNP',
 '서울/NNP-강북구/NNP',
 '강북구/NNP-오/NNP',
 '오/NNP-패사/NNG',
 '패사/NNG-ㄴ/JX',
 'ㄴ/JX-터널/NNP',
 '터널/NNP-인근/NNG',
 '인근/NNG-에서/JKB',
 '에서

이를 CountVectorizer 의 tokenizer 로 입력합니다. 

In [8]:
vectorizer = CountVectorizer(tokenizer=bigram_tokenizer)
x = vectorizer.fit_transform(docs)

총 18,835 개의 uni + bigram 으로 term frequency vector 가 만들어집니다.

In [9]:
x.shape

(965, 18835)

## 4. All bigrams

sklearn.feature_extraction.text.CountVectorizer 의 ngram_range=(1, 2) 로 설정한다면, 가능한 모든 종류의 bigram 을 term 으로 이용하게 됩니다.

In [10]:
from sklearn.feature_extraction.text import CountVectorizer

def all_bigram_tokenizer(sent):
    if not sent:
        return []
    return komoran.pos(sent, join=True)

vectorizer_ = CountVectorizer(
    ngram_range=(1,2),
    tokenizer=all_bigram_tokenizer
)

x_ = vectorizer_.fit_transform(docs)

109,481 개의 uni + bigram 을 이용하여 term frequency vector 를 만듭니다. ngram_range 를 이용할 때에는 min_df, max_df 를 이용하여 infrequent bigram 을 제거해야 적절한 크기의 차원을 유지할 수 있습니다.

In [11]:
x_.shape

(965, 109481)